# Natural Gas Pricing Model

##   Parameters:
  1. injection_dates (list of str): Dates when gas is injected.
  2. withdrawal_dates (list of str): Dates when gas is withdrawn.
  3. prices (dict): A dictionary with dates as keys and prices as values.
  4. injection_rate (float): Rate at which gas is injected (units per day).
  5. withdrawal_rate (float): Rate at which gas is withdrawn (units per day).
  6. max_volume (float): Maximum volume that can be stored (units).
  7. storage_cost_per_month (float): Monthly storage cost.
  8. injection_cost (float): Cost per unit of gas injected.
  9. withdrawal_cost (float): Cost per unit of gas withdrawn.
  10. transport_cost (float): Fixed cost for each injection/withdrawal.


In [1]:
import pandas as pd

In [5]:
# Function to load the price data from the Excel file
def load_price_data(file_path):
    # Read the Excel file
    data = pd.read_csv(file_path)
    
    # Convert the 'Date' column to datetime format
    data['Dates'] = pd.to_datetime(data['Dates'])
    
    # Create a dictionary of prices with dates as keys and prices as values
    price_data = pd.Series(data['Prices'].values, index=data['Dates']).to_dict()
    
    return price_data

In [6]:
def calculate_contract_value(
    injection_dates, withdrawal_dates, prices, injection_rate, withdrawal_rate, 
    max_volume, storage_cost_per_month, injection_cost, withdrawal_cost, transport_cost
):
   
    # Convert dates to datetime
    injection_dates = pd.to_datetime(injection_dates)
    withdrawal_dates = pd.to_datetime(withdrawal_dates)

    # Track volume in storage
    total_volume = 0 
    total_revenue = 0
    total_expense = 0
    total_storage_cost = 0

    # Handle injections
    for date in injection_dates:
        if date in prices:
            injection_price = prices[date]
            inject_volume = min(injection_rate, max_volume - total_volume)  # Ensure no overflow
            total_volume += inject_volume
            total_expense += inject_volume * (injection_price + injection_cost)  # Add injection cost
            total_expense += transport_cost  # Add transport cost for injection
        else:
            raise ValueError(f"No price data available for injection date: {date}")

    # Handle withdrawals
    for date in withdrawal_dates:
        if date in prices:
            withdrawal_price = prices[date]
            withdraw_volume = min(withdrawal_rate, total_volume)  # Ensure no underflow
            total_volume -= withdraw_volume
            total_revenue += withdraw_volume * (withdrawal_price - withdrawal_cost)  # Subtract withdrawal cost
            total_expense += transport_cost  # Add transport cost for withdrawal
        else:
            raise ValueError(f"No price data available for withdrawal date: {date}")

    # Calculate storage cost
    duration_months = (withdrawal_dates[-1] - injection_dates[0]).days / 30
    total_storage_cost = duration_months * storage_cost_per_month

    # Net value of the contract
    contract_value = total_revenue - total_expense - total_storage_cost
    return contract_value


In [10]:

# Example Usage
if __name__ == "__main__":
    # Example price data (date: price)
    file_name = "Nat_Gas.csv"
    price_data = load_price_data(file_name)

    # Inputs
    injection_dates = ['2021-05-31', '2021-06-30', '2021-07-31']
    withdrawal_dates = ['2022-01-31']
    injection_rate = 1e6  # 1 million MMBtu/day
    withdrawal_rate = 1e6  # 1 million MMBtu/day
    max_volume = 1e6  # Maximum 1 million MMBtu in storage
    storage_cost_per_month = 100000  # $100K/month
    injection_cost = 0.01  # $10K per million MMBtu ($0.01/MMBtu)
    withdrawal_cost = 0.01  # $10K per million MMBtu ($0.01/MMBtu)
    transport_cost = 50000  # $50K fixed cost per injection/withdrawal

    # Calculate contract value
    contract_value = calculate_contract_value(
        injection_dates, withdrawal_dates, price_data, injection_rate, 
        withdrawal_rate, max_volume, storage_cost_per_month, 
        injection_cost, withdrawal_cost, transport_cost
    )
    print(f"Net Value of the Contract: ${contract_value:,.2f}")

Net Value of the Contract: $623,333.33


C:\Users\User\AppData\Local\Temp\ipykernel_2648\2138249427.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Dates'] = pd.to_datetime(data['Dates'])
